In [ ]:
cur_date="2023-03-10"
filter_value = "包装印刷,中药"
start_value="10"
end_value="100"


In [ ]:
import pandas as pd
df = pd.read_csv(
    "data/hist.csv", parse_dates=['日期'], index_col=0, dtype={"股票编号":object})
# df.drop("Unnamed: 0", inplace=True, axis=1)
df.info()

In [ ]:
code_df = pd.read_csv("data/板块名称_股票对应.csv", index_col=0)
code_dict = code_df.set_index("代码")['板块名称'].to_dict()
# code_dict


In [ ]:
data=code_dict.get("603858")
data

In [ ]:
df['板块名称'] = df["股票编号"].apply(lambda x:code_dict.get(x))


In [ ]:
df.tail()

In [ ]:
df[df['成交额'] > int(start_value)*100_000_000]


In [ ]:
max=df['成交额'].max()
num_in_yuan = round(max / 100000000, 2)
num_str = '{:.2f} 亿元'.format(num_in_yuan)
print(num_str)


In [ ]:
# df['涨跌幅'] = df['收盘价'].pct_change()
cur_df = df.loc[cur_date]
_list=filter_value.split(",")
cur_df = cur_df[~cur_df['板块名称'].isin(_list)]
cur_df = cur_df[(cur_df['成交额'] >= int(start_value)*100_000_000)
                & (cur_df['成交额'] <= int(end_value)*100_000_000)]
cur_df


In [ ]:
df.info()


In [ ]:
null_rows = df[df.isnull().any(axis=1)]
null_rows


In [ ]:
cur_df.isnull().sum()

In [ ]:
cur_df.dropna(inplace=True)

In [ ]:
import plotly.express as px
fig = px.treemap(cur_df, path=[px.Constant('All'), '板块名称'], values='成交额', height=1080, width=1920,
                 color='涨跌幅', color_continuous_scale='Geyser', range_color=[-0.05, 0.05], color_continuous_midpoint=0,
                 hover_data={"涨跌幅": ':.2%', "成交额": ':.2f'})
fig.update_traces(textinfo="label+value", textfont=dict(size=24))
fig.show()


In [ ]:
# import plotly.express as px
# fig = px.treemap(df, path=[px.Constant('All'), '板块名称'], values='成交额', height=1080, width=1920,
#                  color='涨跌幅', color_continuous_scale='Geyser', range_color=[-0.05, 0.05], color_continuous_midpoint=0,
#                  hover_data={"涨跌幅": ':.2%', "成交额": ':.2f'})
# fig.update_traces(textinfo="label+value", textfont=dict(size=24))
# fig.show()


In [ ]:
# 按股票名称分组，并统计涨幅大于0和小于0的股票数量
result = cur_df.groupby('板块名称')['涨跌幅'].agg(
    [('涨的数量', lambda x: sum(x > 0)), ('跌的数量', lambda x: sum(x < 0))])
result


In [ ]:
result.loc[['IT服务', 'LED']]


In [ ]:
import plotly.graph_objs as go

# 将结果转换为 Pandas DataFrame
# result_df = result.loc[['IT服务', 'LED']].reset_index()

## 获得前十条数据
result_df = result.iloc[:10].reset_index()

# result_df = result.iloc[:10].reset_index()

# 绘制柱状图
fig = go.Figure(data=[
    go.Bar(name='涨的数量', x=result_df['板块名称'], y=result_df['涨的数量']),
    go.Bar(name='跌的数量', x=result_df['板块名称'], y=result_df['跌的数量'])
])

# 设置图表布局和标题
fig.update_layout(title='涨跌幅统计图', xaxis_title='板块名称', yaxis_title='数量')

# 显示图表
fig.show()


In [ ]:
def round_age(code):
    return result.loc[code]['涨的数量']


def unround_age(code):
    return result.loc[code]['跌的数量']


cur_df['涨的数量'] = cur_df['板块名称'].apply(lambda row: round_age(row))
cur_df['跌的数量'] = cur_df['板块名称'].apply(lambda row: unround_age(row))


In [ ]:
cur_df.head()


In [ ]:
import plotly.express as px
fig = px.treemap(cur_df, path=[px.Constant('All'), '板块名称'], values='成交额', height=1080, width=1920,
                 color='涨跌幅', color_continuous_scale='Geyser', range_color=[-0.05, 0.05], color_continuous_midpoint=0,
                  )
# fig.update_traces(textinfo="label+value", textfont=dict(size=24))
fig.update_traces(hovertemplate='Name: %{x}<br>Age: %{y}<br>Rounded Age: %{customdata[0]}',
                  customdata=df.apply(lambda row: [round_age(row['板块名称'])], axis=1))
fig.show()


In [ ]:
import plotly.express as px
fig = px.treemap(cur_df, path=[px.Constant('All'), '板块名称'], values='成交额', height=1080, width=1920,
                 color='涨跌幅', color_continuous_scale='Geyser', range_color=[-0.05, 0.05], color_continuous_midpoint=0,
                 hover_data={"涨跌幅": ':.2%', "成交额": ':.2f', "涨的数量": ":,", "跌的数量":":,"})
fig.update_traces(textinfo="label+value", textfont=dict(size=24))
fig.show()
